# **JL-FreeTime: Optimización y Análisis del Uso del Tiempo Libre** 

### **En este archivo ipybn se hace el diseño de una base de datos *SQL* enfocada en registrar la cantidad de tiempo invertida en actividades recreativas, con el fin de analizar el uso del tiempo libre y el progreso en tales actividades.**

-La base de datos está creada a partir de la librería de Pyhton  ***sqlite3***. 

-El diseño está basado en las actividades que yo realizo. El código puede adaptarse a las actividades de cualquier usuario. 

## ***1. Construcción de la Base de datos***

In [1]:
# Librerías necesarias:
import pandas as pd 
import sqlite3

In [2]:
# Creación y conexión de la base de datos
conn = sqlite3.connect('JL-FreeTime.db')

Para evitar escribir el código para abrir y cerrar los cursores en cada consulta, creamos la siguiente función: 

In [3]:
def queries_function(query, commit=False):
	""" 
	Función para crear tablas.

	query:  Código SQL para generar la tabla. 
	commit: Indica que debe hacerse un commit cuando toma al valor True.
	"""

	# Abrir cursor
	cursor = conn.cursor()

	# Ejecución de la consulta
	cursor.execute(query)

	# Se hace commit de ser necesario
	if commit == True:
		conn.commit()
		
	# Cerrar cursor
	cursor.close()

La tabla principal de la base de datos será la que registre la fecha. Se contemplan los campos **Año**, **Mes** y **Día** por separado con el fin de poder agrupar de manera directa cuando se quiera hacer un análisis:

In [4]:
# Consulta para crear la tabla
query = """ 
		CREATE TABLE "Fecha" (
		"Id_fecha"	INTEGER,
		"Año"	INTEGER,
		"Mes"	INTEGER,
		"Día"	INTEGER,
		PRIMARY KEY("Id_fecha" AUTOINCREMENT)
	);
	"""
queries_function(query=query)

# Ponemos una fecha de prueba:
query = ''' 
	INSERT INTO Fecha (Año, Mes, Día)
    VALUES(2024, 11, 12)
'''
queries_function(query=query, commit=True)

Corroboramos la creación de la tabla (y la de todas la tablas generadas a lo largo de este trabajo) extrayendo su información con la siguiente consulta:

In [5]:

# Nombre de la tabla:
nombre_tabla = 'Fecha'

# Obtener información de la tabla:
query = f''' 
    PRAGMA table_info({nombre_tabla});
'''
# Guardar e imprimir la información de la tabla:
Tabla_info = pd.read_sql_query(query, conn)
Tabla_info

,cid,name,type,notnull,dflt_value,pk
0,0,Id_fecha,INTEGER,0,None,1
1,1,Año,INTEGER,0,None,0
2,2,Mes,INTEGER,0,None,0
3,3,Día,INTEGER,0,None,0


También podemos observarla directamente:

In [6]:
query = ''' 
    SELECT * FROM Fecha
'''
tabla_info = pd.read_sql_query(query, conn)
tabla_info  

,Id_fecha,Año,Mes,Día
0,1,2024,11,12


### ***1.1 Música***

Un hobby que podemos registrar es la ***Música***. A grandes rasgos, se puede estudiar la teoría, practicar una canción o aprender una canción. 

Para registrar el tiempo invertido en la música se crea la siguiente tabla: 

In [7]:
query = ''' 
    CREATE TABLE "Música" (
	"Id_música"	INTEGER,
	"Id_fecha"	INTEGER,
	"Teoría"	INTEGER DEFAULT 0,
	"Práctica"	INTEGER DEFAULT 0,
	"Aprender_canción"	INTEGER DEFAULT 0,
	FOREIGN KEY("Id_fecha") REFERENCES "Fecha"("Id_fecha"),
	PRIMARY KEY("Id_música" AUTOINCREMENT)
);
'''
queries_function(query=query)

# Ponemos el registro de prueba:
query = ''' 
	INSERT INTO Música (Id_fecha)
    VALUES(1)
'''
queries_function(query=query, commit=True)

Donde basta tomar la suma sobre los campos ***Teoría***, ***Práctica***, ***Aprender canción*** para obtener el tiempo destinado a la música. 

Se coloca el registro de prueba debido a que la rutina para realizar un registro (que se proporciona en el repositorio de Github), consulta el Id del último registro realizado. Si el regsitro de prueba no es colocado, cuando el usuario realice su primer registro surgirá un error, pues no se encontrará un último registro. Puesto que los campos numéricos cuentan con 0 como valor por default, esto no afecta en absoluto. Por tal motivo algunas tablas contarán con un registro de prueba o con registros de ejemplos. 

Para registrar el progreso, consideramos los detalles de la práctica y el aprendizaje en cada sesión de música (identificado por ***Id_música***): 

In [8]:
query = ''' 
    CREATE TABLE "Aspectos" (
	"Id_aspecto"	INTEGER,
	"Aspecto"	TEXT,
	PRIMARY KEY("Id_aspecto" AUTOINCREMENT)
);
'''
queries_function(query=query)

# Se agregan los aspectos:
query = ''' 
    INSERT INTO Aspectos (Aspecto)
    VALUES ('Teoría'),
           ('Práctica'),
           ('Aprender_Nuevo');
'''
queries_function(query=query, commit=True)

Se pueden registra la información de las canciones aprendidas o practicadas mediante el registro de la información de las bandas que las tocan:

In [9]:
query = ''' 
    CREATE TABLE "Bandas" (
	"Id_banda"	INTEGER,
	"Nombre_banda"	TEXT,
	"Género"	TEXT,
	"País"	TEXT,
	PRIMARY KEY("Id_banda" AUTOINCREMENT)
);
'''
queries_function(query=query)

# Se agregan algunas bandas:
query = ''' 
    INSERT INTO Bandas (Nombre_banda, Género, País)
	VALUES ('Dream_Theater','Prog_Metal','USA'),
		   ('Liquid_Tension_Experiment','Prog_Metal','USA'),
	       ('Molotov','Punk','México'),
	       ('Los_Búnkers','Rock','Chile'),
           ('Avenged_Sevenfold', 'Heavy_Metal', 'USA');
'''
queries_function(query=query, commit=True)

Así como la información de dichas canciones:

In [10]:
query = ''' 
    CREATE TABLE "Canciones" (
	"Id_canción"	INTEGER,
	"Id_banda"	INTEGER,
	"Nombre_canción"	TEXT,
	"Duración"	INTEGER,
    "Completado"	INTEGER DEFAULT 0,
	FOREIGN KEY("Id_banda") REFERENCES "Bandas"("Id_banda"),
	PRIMARY KEY("Id_canción" AUTOINCREMENT)
);
'''
queries_function(query=query)

# Se agragan algunas canciones:
query = ''' 
	INSERT INTO Canciones (Id_banda, Nombre_canción, Duración)
	VALUES (1, 'Stream_of_consciousness', 11),
		   (5, 'Afterlife', 4),
		   (2, 'Paradigma_shift', 6)
'''
queries_function(query=query, commit=True)

El campo ***Completado*** toma valores 1 o 0 (si o no) y hace referencia a si ya se memorizaron todas las notas de la canción. La duración es en minutos.

Con lo anterior se genera una tabla con los detalles de cada sesión de música:

In [11]:
query= '''
    CREATE TABLE "Música_detalles" (
	"Id_música_detalles"	INTEGER,
	"Id_música"	INTEGER,
	"Id_canción"	INTEGER,
	"Id_aspecto"	INTEGER,
	FOREIGN KEY("Id_canción") REFERENCES "Canciones"("Id_canción"),
	FOREIGN KEY("Id_aspecto") REFERENCES "Aspectos"("Id_aspecto"),
	FOREIGN KEY("Id_música") REFERENCES "Música"("Id_música"),
	PRIMARY KEY("Id_música_detalles" AUTOINCREMENT)
);
'''
queries_function(query=query)

Corroboramos que se haya registrado la información:

In [12]:
query = ''' 
    SELECT * FROM Canciones
'''
Tabla_info = pd.read_sql_query(query, conn)
Tabla_info

,Id_canción,Id_banda,Nombre_canción,Duración,Completado
0,1,1,Stream_of_consciousness,11,0
1,2,5,Afterlife,4,0
2,3,2,Paradigma_shift,6,0


### ***1.2 Actividad Física*** 

Para registrar el tiempo invertido en hacer ejercicio consideramos por separado el tiempo invertido en ejercicios de fuerza, flexibilidad, resistencia, técnica (si se practica alguna disciplina en particular) etc.: 

In [13]:
query = ''' 
    CREATE TABLE "Actividad_física" (
	"Id_actividad_física"	INTEGER,
	"Id_fecha"	INTEGER,
	"Fuerza"	INTEGER DEFAULT 0,
	"Flexibilidad"	INTEGER DEFAULT 0,
	"Resistencia"	INTEGER DEFAULT 0,
	"Técnica"	INTEGER DEFAULT 0,
	FOREIGN KEY("Id_fecha") REFERENCES "Fecha"("Id_fecha"),
	PRIMARY KEY("Id_actividad_física" AUTOINCREMENT)
);
'''
queries_function(query=query)

# Registro de prueba:
query = ''' 
	INSERT INTO Actividad_física (Id_fecha)
    VALUES(1)
'''
queries_function(query=query, commit=True)

Así, el tiempo invertido es la suma de los tiempos en cada campo (que no es un id) de la tabla anterior. 

Para monitorear el progreso, consideramos ejercicios representativos como abdominales, sentadillas, lagartijas y una rutina de cardio o resistencia, y registramos el número máximo de repeticiones sin parar (primeros tres ejercicios) y el tiempo máximo sin parar (rutina de resistencia) en segundos. 

In [14]:
query = ''' 
    CREATE TABLE "Progreso" (
	"Id_progreso"	INTEGER,
	"Id_actividad_física"	INTEGER,
	"Abdominales"	INTEGER DEFAULT 0,
	"Sentadillas"	INTEGER DEFAULT 0,
	"Lagartijas"	INTEGER DEFAULT 0,
	"Rutina_de_resistencia"	REAL DEFAULT 0,
	PRIMARY KEY("Id_progreso" AUTOINCREMENT),
	FOREIGN KEY("Id_actividad_física") REFERENCES "Actividad_física"("Id_actividad_física")
);
'''
queries_function(query=query)

# Registro de prueba:
query = ''' 
	INSERT INTO Progreso (Id_actividad_física)
    VALUES(1)
'''
queries_function(query=query, commit=True)

### ***1.3 Estudio***

El tiempo de estudio (fuera del realizado en la escuela o el trabajo) puede registrarse considerando el tiempo dedicado a cierta área de estudio:

In [15]:
query = ''' 
    CREATE TABLE "Estudio" (
	"Id_estudio"	INTEGER,
	"Id_fecha"	INTEGER,
	"Física"	INTEGER DEFAULT 0,
	"Matemáticas"	INTEGER DEFAULT 0,
	"Ciencias_Biológicas"	INTEGER DEFAULT 0,
	"Otro"	INTEGER DEFAULT 0,
	PRIMARY KEY("Id_estudio" AUTOINCREMENT),
	FOREIGN KEY("Id_fecha") REFERENCES "Fecha"("Id_fecha")
);
''' 
queries_function(query=query)

# Registro de prueba:
query = ''' 
	INSERT INTO Estudio (Id_fecha)
    VALUES(1)
'''
queries_function(query=query, commit=True)

Nuevamente, el tiempo de estudio es la suma de los tiempos registrados en la tabla. Los detalles de cada sesión de estudio se registran a partir de las materias estudiadas:

In [16]:
query = ''' 
    CREATE TABLE "Ramas" (
	"Id_rama"	INTEGER,
	"Nombre_rama"	TEXT,
	PRIMARY KEY("Id_rama" AUTOINCREMENT)
);  
'''
queries_function(query=query)

# Se agregan algunas materias:
query = ''' 
    INSERT INTO Ramas (Nombre_rama)
    VALUES ('Física'),
           ('Matemáticas'),
           ('Ciencias Biológicas'),
           ('Finanzas/Economía');
'''
queries_function(query=query, commit=True)

query = ''' 
    CREATE TABLE "Materias" (
	"Id_materia"	INTEGER,
    "Id_rama"	INTEGER,
	"Nombre_materia"	TEXT,
	PRIMARY KEY("Id_materia" AUTOINCREMENT)
    FOREIGN KEY("Id_rama") REFERENCES "Ramas"("Id_rama")
);  
'''
queries_function(query=query)

# Se agregan algunas materias:
query = ''' 
    INSERT INTO Materias (Nombre_materia, Id_rama)
    VALUES ('Termodinámica', 1),
           ('Cálculo Diferencial', 2),
           ('Cálculo Integral', 2),
           ('Cálculo Diferencial de varias variables', 2),
           ('Cálculo Integral de varias variables', 2),
           ('Probabilidad y estadística', 2),
           ('Mecánica', 1),
           ('Relatividad', 1);
'''
queries_function(query=query, commit=True)

Se puede registrar también el tiempo dedicado a cada materia:

In [17]:
query = ''' 
    CREATE TABLE "Estudio_detalles" (
	"Id_estudio_detalles"	INTEGER,
	"Id_estudio"	INTEGER,
	"Id_materia"	INTEGER,
	FOREIGN KEY("Id_materia") REFERENCES "Materias"("Id_materia"),
	FOREIGN KEY("Id_estudio") REFERENCES "Estudio"("Id_estudio"),
	PRIMARY KEY("Id_estudio_detalles" AUTOINCREMENT)
); 
'''
queries_function(query=query)

### ***1.4 Programación***

Para registrar el tiempo usado en programación consideramos el invertido en la teoría y el invertido en la práctica:

In [18]:
query = ''' 
    CREATE TABLE "Programación" (
	"Id_programación"	INTEGER,
	"Id_fecha"	INTEGER,
	"Teoría"	INTEGER DEFAULT 0,
	"Práctica"	INTEGER DEFAULT 0,
	PRIMARY KEY("Id_programación" AUTOINCREMENT),
    FOREIGN KEY("Id_fecha") REFERENCES "Fecha"("Id_fecha")
);
'''
queries_function(query=query)

# Registro de prueba:
query = ''' 
	INSERT INTO Programación (Id_fecha)
    VALUES(1)
'''
queries_function(query=query, commit=True)

En este caso consideramos los lenguajes de programación usados:

In [19]:
query = ''' 
    CREATE TABLE "Lenguajes" (
	"Id_lenguaje"	INTEGER,
	"Nombre_lenguaje"	TEXT,
	PRIMARY KEY("Id_lenguaje" AUTOINCREMENT)
);
'''
queries_function(query=query)

# Se agregan algunos lenguajes: 
query = ''' 
    INSERT INTO Lenguajes (Nombre_lenguaje)
    VALUES ('Python'),
           ('SQL'),
           ('HTML + CSS + JS');
'''
queries_function(query=query, commit=True)

Con lo cual se registra el tiempo invertido en practicar o aprender un lenguaje en específico:

In [20]:
query = ''' 
    CREATE TABLE "Programación_detalles" (
	"Id_programación_detalles"	INTEGER,
	"Id_programación"	INTEGER,
	"Id_lenguaje"	INTEGER,
	"Id_aspecto"	INTEGER,
	"Tiempo"	INTEGER DEFAULT 0,
    FOREIGN KEY("Id_aspecto") REFERENCES "Aspectos"("Id_aspecto"),
	FOREIGN KEY("Id_lenguaje") REFERENCES "Lenguajes"("Id_lenguaje"),
	PRIMARY KEY("Id_programación_detalles" AUTOINCREMENT)
);
'''
queries_function(query=query)

### ***1.5 Lectura***

Se registran los libros que se están leyendo:



In [21]:
query = '''
    CREATE TABLE "Libros" (
	"Id_libro"	INTEGER,
	"Nombre_libro"	TEXT,
	"Autor"	TEXT,
	"Completado"	INTEGER DEFAULT 0,
	PRIMARY KEY("Id_libro" AUTOINCREMENT)
);
'''
queries_function(query=query)

# Se agrega un libro:
query = '''
    INSERT INTO Libros (Nombre_libro, Autor)
    VALUES ('Azteca', 'Gary Jennings');
'''
queries_function(query=query, commit=True)

A partir de la cual creamos una tabla que registra el tiempo y el líbro leído cada día:

In [22]:
query = ''' 
    CREATE TABLE "Lectura" (
	"Id_lectura"	INTEGER,
	"Id_fecha"	INTEGER,
	"Id_libro"	INTEGER,
	"Tiempo"	INTEGER DEFAULT 0,
	FOREIGN KEY("Id_libro") REFERENCES "Libros"("Id_libro"),
	FOREIGN KEY("Id_fecha") REFERENCES "Fecha"("Id_fecha"),
	PRIMARY KEY("Id_lectura" AUTOINCREMENT)
);
'''
queries_function(query=query)

# Registro de prueba:
query = ''' 
	INSERT INTO Lectura (Id_fecha)
    VALUES(1)
'''
queries_function(query=query, commit=True)

### ***1.6 Dibujo***

Creamos una tabla para registrar el tiempo invertido en la teoría o  práctica del dibujo:

In [23]:
query = ''' 
    CREATE TABLE "Dibujo" (
	"Id_dibujo"	INTEGER,
	"Id_fecha"	INTEGER,
	"Teoría"	INTEGER DEFAULT 0,
	"Práctica"	INTEGER DEFAULT 0,
	FOREIGN KEY("Id_fecha") REFERENCES "Fecha"("Id_fecha"),
	PRIMARY KEY("Id_dibujo" AUTOINCREMENT)
);
'''
queries_function(query=query)

# Registro de prueba:
query = ''' 
	INSERT INTO Dibujo (Id_fecha)
    VALUES(1)
'''
queries_function(query=query, commit=True)

registramos los dibujos hechos en caso de práctica y los temas estudiados en caso de estudiar la teoría, se registra también si el dibujo se terminó o si el tema se vió completamente (completado = 1 o 0)

In [24]:
query = ''' 
    CREATE TABLE "Obras" (
	"Id_obra"	INTEGER,
	"Título"	TEXT,
	"Técnica"	TEXT,
	"Completado"	INTEGER DEFAULT 0,
	PRIMARY KEY("Id_obra" AUTOINCREMENT)
);
'''
queries_function(query=query)

# Agregamos una obra:
query = ''' 
	INSERT INTO Obras (Título, Técnica, Completado)
    VALUES('Estudio de teoría', 'Cualquiera', 0)
'''
queries_function(query=query, commit=True)

De este modo se pueden registrar los detalles del tiempo dedicado a la realización de una obra en particular o de un tema en específico:

In [25]:
query = ''' 
    CREATE TABLE "Dibujo_detalles" (
	"Id_dibujo_detalles"	INTEGER,
	"Id_aspecto"	INTEGER,
	"Id_dibujo"	INTEGER,
	"Id_obra"	INTEGER,
	FOREIGN KEY("Id_obra") REFERENCES "Obras"("Id_obra"),
	FOREIGN KEY("Id_aspecto") REFERENCES "Aspectos"("Id_aspecto"),
	PRIMARY KEY("Id_dibujo_detalles" AUTOINCREMENT)
);
'''
queries_function(query=query)

### ***1.6 Idioma***

Finalmente se puede registrar el tiempo invertido en aprender otro idioma. Para esto se considera el tiempo invertido en cada habilidad (Hablar, escribir, leer, escuchar, gramática) y por supuesto, el idioma en cuestión para aquellos políglotas:

In [26]:
query = ''' 
    CREATE TABLE "Idiomas" (
	"Id_idioma"	INTEGER,
	"Idioma"	TEXT,
	PRIMARY KEY("Id_idioma" AUTOINCREMENT)
);
'''
queries_function(query=query)

# Agregamos una Idioma:
query = ''' 
	INSERT INTO Idiomas (Idioma)
    VALUES('Inglés')
'''
queries_function(query=query, commit=True)

query = ''' 
    CREATE TABLE "Aprender_idiomas" (
	"Id_aprender_idiomas"	INTEGER,
	"Id_fecha"	INTEGER,
	"Id_idioma"	INTEGER,
	"Hablar"	INTEGER DEFAULT 0,
	"Escuchar"	INTEGER DEFAULT 0,
	"Escribir"	INTEGER DEFAULT 0,
	"Leer"	INTEGER DEFAULT 0,
	"Gramática"	INTEGER DEFAULT 0,
	FOREIGN KEY("Id_idioma") REFERENCES "Idioma"("Id_idioma"),
	FOREIGN KEY("Id_fecha") REFERENCES "Fecha"("Id_fecha"),
	PRIMARY KEY("Id_aprender_idiomas" AUTOINCREMENT)
);
'''
queries_function(query=query)

# Registro de prueba:
query = ''' 
	INSERT INTO Aprender_idiomas (Id_fecha)
    VALUES(1)
'''
queries_function(query=query, commit=True)

El código escrito hasta ahora genera la siguiente base de datos:
<br>
<br>
<center><div>
<img src="Jl_FreeTimeC.png" width="600"/>
<p> Figura 1: Base de datos JL-FreeTime para monitorear el uso del tiempo libre. <br> Los campos en amarillo corresponden a claves primarias. <br>Los campos en verde representan claves foráneas.</p>
</div></center>
<br>